<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-packages-&amp;-dataset" data-toc-modified-id="Importing-packages-&amp;-dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing packages &amp; dataset</a></span></li><li><span><a href="#Exploring-the-dataset" data-toc-modified-id="Exploring-the-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Exploring the dataset</a></span><ul class="toc-item"><li><span><a href="#General-features" data-toc-modified-id="General-features-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>General features</a></span></li><li><span><a href="#Getting-to-know-the-labels" data-toc-modified-id="Getting-to-know-the-labels-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Getting to know the labels</a></span><ul class="toc-item"><li><span><a href="#Can-a-comment-be-both-toxic-and-severely-toxic?" data-toc-modified-id="Can-a-comment-be-both-toxic-and-severely-toxic?-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Can a comment be both toxic and severely toxic?</a></span></li><li><span><a href="#Do-the-obsene/threat/insult/identity_hate-tags-come-under-the-toxic-umbrella?" data-toc-modified-id="Do-the-obsene/threat/insult/identity_hate-tags-come-under-the-toxic-umbrella?-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Do the obsene/threat/insult/identity_hate tags come under the toxic umbrella?</a></span></li><li><span><a href="#Adding-a-non-toxic-column:" data-toc-modified-id="Adding-a-non-toxic-column:-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Adding a non-toxic column:</a></span></li><li><span><a href="#Removing-the-total_score-column" data-toc-modified-id="Removing-the-total_score-column-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Removing the total_score column</a></span></li></ul></li></ul></li><li><span><a href="#Pre-Modelling:-spaCy" data-toc-modified-id="Pre-Modelling:-spaCy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pre-Modelling: spaCy</a></span></li></ul></div>

# Exploring spaCy: Toxicity Levels 

**Overview**

Using the SpaCy package to explore the [Toxic Comment Classification Challenge](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/overview). 

- Challenge: build a model that’s capable of detecting different types of of toxicity like **threats, obscenity, insults, and identity-based hate**. 
- The dataset contains comments from Wikipedia’s talk page edits. 
- The goal is to help online discussion become more productive and respectful.

**Dataset Description**

A large number of Wikipedia comments which have been labeled by human raters for toxic behavior. 
The types of toxicity are:

- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

You must create a model which predicts a **probability of each type of toxicity** for each comment.

**File descriptions**

- train.csv - the training set, contains comments with their binary labels
- test.csv - the test set, you must predict the toxicity probabilities for these comments.
- sample_submission.csv - a sample submission file in the correct format
- test_labels.csv - labels for the test data; value of -1 indicates it was not used for scoring; (Note: file added after competition close!)

## Importing packages & dataset

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
sns.set(font_scale=1.5)
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [15]:
df = pd.read_csv("/Users/anastasiakuzmich/Desktop/jigsaw-toxic-comment-classification-challenge/train.csv")

print("The dataset contains %s entries with %s features." 
      % (df.shape[0], df.shape[1]))

The dataset contains 159571 entries with 8 features.


## Exploring the dataset

### General features

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


✏️ Okay, there's no missing values & the data types are fine. Within this dataset there is:

1. An id column which might just be easier to drop. 
2. The comment_text column which contains the content of the comment itself
3. The columns indicating the comment's annotation as toxic, severe_toxic, obscene, threat, insult or identity hate. Let's explore these further:

In [18]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


✏️ The labels are binary, so a comment is either of that label, or not of that label. 9% of the comments are toxic, 5% are obscene, 4% are insults, less than 1% are severely toxic, identity theft or a threat. 

Let's find out if these labels are are mutually exclusive + how many non-toxic comments there are.

In [20]:
df['total_score'] = (df['toxic'] 
                     + df["severe_toxic"] 
                     + df["obscene"] 
                     + df["threat"]
                     + df["insult"]
                     + df["identity_hate"])

df['total_score'].value_counts(normalize=True)

0    0.898321
1    0.039857
3    0.026377
2    0.021808
4    0.011030
5    0.002413
6    0.000194
Name: total_score, dtype: float64

✏️ My takeaways at this stage:

- 90% of the comments aren't negative. That's good to know, but we are dealing with severe class imbalance...
- A good amount of comments ticks several boxes, so the labels are not exclusive. Some tick all 6 somehow? (*How angry do you have to be?*)
- The model needs to be capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate. I need to think about how I'll need to restructure the data and what I want my predictors to be.

**Next questions to address:**

1. Can a comment be both toxic and severely toxic? 
2. Are toxic & severe toxic the umbrella categories, then obsene/threat/insult/identity_hate their sub-categories? Restructure and see.
3. Perhaps at this stage it is worth introducing a non-toxic column? So the model would almost predict whether the comments are toxic vs non-toxic vs severely toxic, then *if* it's toxic then what subcategory of toxic?

### Getting to know the labels

#### Can a comment be both toxic and severely toxic?

In [33]:
print("Number of toxic comments:", len(df[df['toxic'] == 1]))
print("Number of severely toxic comments:", len(df[df['severe_toxic'] == 1]))
print("Number of severely toxic and toxic comments:", len(df[(df['toxic'] == 1) & (df['severe_toxic'] == 1)]))

Number of toxic comments: 15294
Number of severely toxic comments: 1595
Number of severely toxic and toxic comments: 1595


✏️ Yes. All severely toxic comments are also toxic, but not all toxic comments are severe. Perhaps it's worth making them exclusive within the dataframe? Could also do it later on after modelling to see if it would improve the score. 

#### Do the obsene/threat/insult/identity_hate tags come under the toxic umbrella?

In [39]:
print("Number of obsene comments that are not toxic:",
      len(df[(df['toxic'] == 0) & (df['obscene'] == 1)]))

print("Number of threatening comments that are not toxic:",
      len(df[(df['toxic'] == 0) & (df['threat'] == 1)]))

print("Number of insulting comments that are not toxic:",
      len(df[(df['toxic'] == 0) & (df['insult'] == 1)]))

print("Number of identity-hateful comments that are not toxic:",
      len(df[(df['toxic'] == 0) & (df['identity_hate'] == 1)]))

Number of obsene comments that are not toxic: 523
Number of threatening comments that are not toxic: 29
Number of insulting comments that are not toxic: 533
Number of identity-hateful comments that are not toxic: 103


✏️ No, apparently they can be mutually exclusive? Which doesn't make complete sense to me. Let's inspect:

In [58]:
for category in ["obscene", "threat", "insult", "identity_hate"]:
    for i in range(2):
        print("Comment ", i+1, ": ", str(category),
              "\n", 
              df[(df['toxic'] == 0) & (df[category] == 1)]["comment_text"].iloc[i], 
              "\n")

Comment  1 :  obscene 
 How do you know he is dead.  Its just his plane that crashed.  Jeezz, quit busting his nuts, folks. 

Comment  2 :  obscene 
 Bleh. I'm all out of ideas. Is there really something wrong with the server or what? The damn thing won't work for me, either Helvetica_font_new.svg (my upload of Segoe UI was actually a PNG to begin with...) 

Comment  1 :  threat 
 Please stop. If you continue to ignore our policies by introducing inappropriate pages to Wikipedia, you will be blocked. 

Comment  2 :  threat 
 "

 Lock Down 

I don't know about the ""Bulgarian BULLSHIT season"", but don't be fooled from where exactly i connect. If i see you editing, or someone else you send, the DDA40X article one more time, you will spend the next few years in prison. I will arrange you a meeting with certain people for suspicions of terrorism. Yes. I can do that. A terror suspect nowadays, you can kiss your young life goodbye. If you file complaints and i'm banned, you and your naked a

✏️ Ummm... these look toxic to me? But I just re-read the challenge rules and I'm expected to classify each of the columns so I'll try not to overthink why these categories are the way they are. I will make an extra column for non-toxic, neutral comments though because in my head that will make the probabilities make sense?

#### Adding a non-toxic column:

In [66]:
def non_toxic_mapper(x):
    if x == 0: 
        return 1
    else:
        return 0

df["non_toxic"] = df["total_score"].map(non_toxic_mapper)
df["non_toxic"].value_counts()

1    143346
0     16225
Name: non_toxic, dtype: int64

#### Removing the total_score column

This column was only used to explore the data, so I'm dropping it before I start the pre-modelling stages. 

In [69]:
df = df.drop("total_score", axis=1)

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


## Pre-Modelling: spaCy